<a href="https://colab.research.google.com/github/lightwsrld/pytorch/blob/main/transformer_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y torch torchvision torchaudio torchtext torchdata

Found existing installation: torch 1.13.1
Uninstalling torch-1.13.1:
  Successfully uninstalled torch-1.13.1
Found existing installation: torchvision 0.14.1
Uninstalling torchvision-0.14.1:
  Successfully uninstalled torchvision-0.14.1
Found existing installation: torchaudio 0.13.1
Uninstalling torchaudio-0.13.1:
  Successfully uninstalled torchaudio-0.13.1
Found existing installation: torchtext 0.14.1
Uninstalling torchtext-0.14.1:
  Successfully uninstalled torchtext-0.14.1
Found existing installation: torchdata 0.5.1
Uninstalling torchdata-0.5.1:
  Successfully uninstalled torchdata-0.5.1


In [ ]:
!pip install torch==2.0.0 torchvision==0.15.1 torchaudio==2.0.1

  Using cached torch-2.0.0-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
  Using cached torchvision-0.15.1-cp310-cp310-manylinux1_x86_64.whl (6.0 MB)
  Using cached torchaudio-2.0.1-cp310-cp310-manylinux1_x86_64.whl (4.4 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1
    Uninstalling torch-1.13.1:
      Successfully uninstalled torch-1.13.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.1
    Uninstalling torchvision-0.14.1:
      Successfully uninstalled torchvision-0.14.1
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.13.1
    Uninstalling torchaudio-0.13.1:
      Successfully uninstalled torchaudio-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.11 requires torch<1.14,>=1.7, but you have torch 2.0.0 which is incompatible.
torchdata 0.5.1

In [1]:
!pip install torchtext==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.8 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.2
    Uninstalling torchtext-0.15.2:
      Successfully uninstalled torchtext-0.15.2


In [1]:
import torch
import torchvision
import torchtext

print(f'torch version: {torch.__version__}')
print(f'torchvision version: {torchvision.__version__}')
print(f'torchtext version: {torchtext.__version__}')
print("cuda version: {}".format(torch.version.cuda))

torch version: 2.0.1+cu118
torchvision version: 0.15.2+cu118
torchtext version: 0.6.0
cuda version: 11.8


In [2]:
import random
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
%%capture
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

In [4]:
import spacy

spacy_en = spacy.load("en_core_web_sm") # 영어 토큰화(tokenization)
spacy_de = spacy.load("de_core_news_sm") # 독일어 토큰화(tokenization)

In [5]:
# 간단히 토큰화(tokenization) 기능 써보기

tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
    print(f"인덱스 {i}: {token.text}")

인덱스 0: I
인덱스 1: am
인덱스 2: a
인덱스 3: graduate
인덱스 4: student
인덱스 5: .


In [ ]:
#@title 영어 및 독일어 토큰화 함수 정의


In [6]:
def tokenize_de(text):

    return [tok.text for tok in spacy_de.tokenizer(text)]        # input 문장의 토큰화 이후에 순서를 뒤집음 -> (X)

def tokenize_en(text):

    return [tok.text for tok in spacy_en.tokenizer(text)]


In [7]:
PAD_WORD = '<blank>' # padding token
UNK_WORD = '<unk>' # unknown token
BOS_WORD = '<s>' # start token
EOS_WORD = '</s>' # end token

SRC = torchtext.data.Field(                                       # Field 함수는 데이터 타입과 이를 텐서로 변환할 지시사항과 함께 저장, 텐서로 표현될 수 있는 텍스트 타입을 처리하고
                                                                    # 각 토큰을 숫자 인덱스로 맵핑하는 vocab 객체가 존재
    tokenize = tokenize_de, lower=True,
    pad_token=PAD_WORD, init_token=BOS_WORD, eos_token=EOS_WORD)

TRG = torchtext.data.Field(
    tokenize = tokenize_en, lower=True,
    pad_token=PAD_WORD, init_token=BOS_WORD, eos_token=EOS_WORD)

In [8]:
!pip install torchtext.legacy

ERROR: Could not find a version that satisfies the requirement torchtext.legacy (from versions: none)
ERROR: No matching distribution found for torchtext.legacy


In [8]:
from torchtext.datasets import Multi30k
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'),
                                                    fields = (SRC, TRG))

print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:03<00:00, 380kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 110kB/s] 


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 106kB/s] 


Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


In [9]:
print(vars(train_data.examples[25])['src'])
print(vars(train_data.examples[25])['trg'])

['eine', 'person', 'in', 'einem', 'blauen', 'mantel', 'steht', 'auf', 'einem', 'belebten', 'gehweg', 'und', 'betrachtet', 'ein', 'gemälde', 'einer', 'straßenszene', '.']
['a', 'person', 'dressed', 'in', 'a', 'blue', 'coat', 'is', 'standing', 'in', 'on', 'a', 'busy', 'sidewalk', ',', 'studying', 'painting', 'of', 'a', 'street', 'scene', '.']


In [10]:
# 영어와 독어의 단어 사전을 생성 -> 언어 도메인이 다르기 때문에 각자 따로 만듦
# 2개 이상 나온 단어들만 사용

SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

print(f"len(SRC): {len(SRC.vocab)}")
print(f"len(TRG): {len(TRG.vocab)}")

len(SRC): 7853
len(TRG): 5893


In [11]:
# 현재 단어 집합의 단어와 맵핑된 고유한 정수를 출력

TRG.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f1c7b5fcaf0>>,
            {'<unk>': 0,
             '<blank>': 1,
             '<s>': 2,
             '</s>': 3,
             'a': 4,
             '.': 5,
             'in': 6,
             'the': 7,
             'on': 8,
             'man': 9,
             'is': 10,
             'and': 11,
             'of': 12,
             'with': 13,
             'woman': 14,
             ',': 15,
             'two': 16,
             'are': 17,
             'to': 18,
             'people': 19,
             'at': 20,
             'an': 21,
             'wearing': 22,
             'shirt': 23,
             'young': 24,
             'white': 25,
             'black': 26,
             'his': 27,
             'while': 28,
             'blue': 29,
             'men': 30,
             'red': 31,
             'sitting': 32,
             'girl': 33,
             'boy': 34,
             'dog': 35,
             'standin

In [12]:
print(TRG.vocab.stoi["abcabc"]) # 없는 단어: 0
print(TRG.vocab.stoi[TRG.pad_token]) # 패딩(padding): 1
print(TRG.vocab.stoi["<s>"]) # <sos>: 2
print(TRG.vocab.stoi["</s>"]) # <eos>: 3
print(TRG.vocab.stoi["water"])
print(TRG.vocab.stoi["world"])

0
1
2
3
47
1752


In [ ]:
#@title **


In [13]:
# BucketIterator 생성(파이토치 데이터 로더 역할 수행) , 미니 배치 안의 문장들을 같은 크기로 맞춰주는 기능

from torchtext.data import Field, BucketIterator

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

# 일반적인 데이터 로더(data loader)의 iterator와 유사하게 사용 가능
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device)

print(next(iter(train_iterator)).src)
print(next(iter(train_iterator)).src.shape)

tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [   8,   54,    5,  ...,    8,    5,    5],
        [  16, 1551,  717,  ...,   36,   13,   13],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]], device='cuda:0')
torch.Size([33, 128])


In [14]:
for i, batch in enumerate(train_iterator):
    src = batch.src
    trg = batch.trg

    print(f"첫 번째 배치 크기: {src.shape}")  # (batch_size, sequence_length)

    # 현재 배치에 있는 하나의 문장에 포함된 정보 출력
    for i in range(src.shape[1]):
        print(f"인덱스 {i}: {src[0][i].item()}") # 여기에서는 [Seq_num, Seq_len]

    # 첫 번째 배치만 확인
    break

첫 번째 배치 크기: torch.Size([34, 128])
인덱스 0: 2
인덱스 1: 2
인덱스 2: 2
인덱스 3: 2
인덱스 4: 2
인덱스 5: 2
인덱스 6: 2
인덱스 7: 2
인덱스 8: 2
인덱스 9: 2
인덱스 10: 2
인덱스 11: 2
인덱스 12: 2
인덱스 13: 2
인덱스 14: 2
인덱스 15: 2
인덱스 16: 2
인덱스 17: 2
인덱스 18: 2
인덱스 19: 2
인덱스 20: 2
인덱스 21: 2
인덱스 22: 2
인덱스 23: 2
인덱스 24: 2
인덱스 25: 2
인덱스 26: 2
인덱스 27: 2
인덱스 28: 2
인덱스 29: 2
인덱스 30: 2
인덱스 31: 2
인덱스 32: 2
인덱스 33: 2
인덱스 34: 2
인덱스 35: 2
인덱스 36: 2
인덱스 37: 2
인덱스 38: 2
인덱스 39: 2
인덱스 40: 2
인덱스 41: 2
인덱스 42: 2
인덱스 43: 2
인덱스 44: 2
인덱스 45: 2
인덱스 46: 2
인덱스 47: 2
인덱스 48: 2
인덱스 49: 2
인덱스 50: 2
인덱스 51: 2
인덱스 52: 2
인덱스 53: 2
인덱스 54: 2
인덱스 55: 2
인덱스 56: 2
인덱스 57: 2
인덱스 58: 2
인덱스 59: 2
인덱스 60: 2
인덱스 61: 2
인덱스 62: 2
인덱스 63: 2
인덱스 64: 2
인덱스 65: 2
인덱스 66: 2
인덱스 67: 2
인덱스 68: 2
인덱스 69: 2
인덱스 70: 2
인덱스 71: 2
인덱스 72: 2
인덱스 73: 2
인덱스 74: 2
인덱스 75: 2
인덱스 76: 2
인덱스 77: 2
인덱스 78: 2
인덱스 79: 2
인덱스 80: 2
인덱스 81: 2
인덱스 82: 2
인덱스 83: 2
인덱스 84: 2
인덱스 85: 2
인덱스 86: 2
인덱스 87: 2
인덱스 88: 2
인덱스 89: 2
인덱스 90: 2
인덱스 91: 2
인덱스 92: 2
인덱스 93: 2
인덱스 94: 2
인덱스 95: 2
인덱스 96: 2
인덱스 97

In [15]:
batch.trg[:, 1]

tensor([  2,   4,   9,   6,   4, 178,  81,  89,  71,  18,   4,   9,  73,   6,
         61, 780,  40,  69,   4, 265,   5,   3,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1], device='cuda:0')

In [15]:
# TRG.vocab.itos를 통해 토큰 인덱스를 해당 문자열 표현으로 반환 -> source 및 target sequences의 길이 및 크기를 출력
# 최대 길이에 맞춰 padding이 이루어짐

for i, batch in enumerate(train_iterator):
    print(batch.trg.size())
    items = [" ".join([TRG.vocab.itos[item] for item in batch.trg[:, i]]) for i in range(128)]
    for item in items:
        print(item)

    print(batch.src.size())
    items = [" ".join([SRC.vocab.itos[item] for item in batch.src[:, i]]) for i in range(128)]
    for item in items:
        print(item)
    break

torch.Size([34, 128])
<s> man in <unk> <unk> logo shirt laying on a blue blanket , with arm around a toy - chewing infant . </s> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank>
<s> two young men perform martial arts in front of a basketball hoop . </s> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank>
<s> a man on a bridge holds a blue object over the rail . </s> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank>
<s> a worker <unk> the beams of a building being constructed . </s> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank> <blank>
<s> hundreds of people are running a marathon through a city . </s> <blank> <blank> <blank> <blank> <bla

In [ ]:
!pip install dill

In [ ]:
import dill as pickle
import easydict

opt = easydict.EasyDict({
        "lang_src" : 'de_core_news_sm',# source 언어 / spacy 사용시
        "lang_trg" : 'en_core_web_sm', # target 언어 / spacy 사용시
        "save_data" : True,
        "data_src" : None,
        "data_trg" : None,
        "max_len" :100, # 한 문장에 들어가는 최대 token 수 / 해당 갯수 이상이면 버림
        "min_word_count" : 2, # vocab을 만들어 줄때 최소 갯수 : 해당 갯수 미만이면 <unk>로 vocab이 형성됨
        "keep_case": True, # 대소문자 구분
        "share_vocab" : "store_true",  # target vocab과 source vocab을 하나로 합치는지 여부
    })


In [16]:
for w, _ in SRC.vocab.stoi.items():
    if w not in TRG.vocab.stoi:
        TRG.vocab.stoi[w] = len(TRG.vocab.stoi)
TRG.vocab.itos = [None] * len(TRG.vocab.stoi)
for w, i in TRG.vocab.stoi.items():
    TRG.vocab.itos[i] = w
SRC.vocab.stoi = TRG.vocab.stoi
SRC.vocab.itos = TRG.vocab.itos
print('[Info] Get merged vocabulary size:', len(TRG.vocab))

[Info] Get merged vocabulary size: 13269


In [17]:
data = {
    'vocab': {'src': SRC, 'trg': TRG},
    'train': train_data.examples,
    'valid': valid_data.examples,
    'test': test_data.examples}


In [ ]:
#@title 1. Positional Encoding

In [18]:
import torch.nn as nn
vacab_len = 13269

embedding_layer = nn.Embedding(num_embeddings=vacab_len,
                               embedding_dim=512,
                               padding_idx=1)

print(f'LOOK UP TABLE SIZE:  {embedding_layer.weight.shape}')
print(embedding_layer.weight)

LOOK UP TABLE SIZE:  torch.Size([13269, 512])
Parameter containing:
tensor([[-0.1117, -0.4966,  0.1631,  ...,  0.9230, -0.6661, -0.6180],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.5411,  0.8900, -0.2705,  ..., -1.0925, -0.7953,  2.1657],
        ...,
        [ 0.2053, -0.2856,  1.3139,  ...,  2.4629, -1.6645, -1.5972],
        [ 2.0385,  0.7906, -2.6038,  ..., -0.9524, -0.8998,  1.2526],
        [-0.9399, -0.3463, -1.2625,  ..., -1.9454,  1.2405, -0.9553]],
       requires_grad=True)


In [19]:
import torch.nn as nn
import numpy as np

class Positional_Encoding(nn.Module):
    def __init__(self, d_hid, max_length=200):
        super(Positional_Encoding, self).__init__()

        self.register_buffer('pos_table', self._get_sinusoid_encoding_table(max_length, d_hid))

    def _get_sinusoid_encoding_table(self, max_length, d_hid):
        '''Sinusoid position encoding table'''

        def get_position_angle_vec(position):
            return [position / np.power(10000, 2 * (hid_j // 2) / d_hid) for hid_j in range(d_hid)]

        sinusoid_table = np.array([get_position_angle_vec(pos_i) for pos_i in range(max_length)])
        sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])     # 짝수 번째 인덱스는 sin 함수 적용
        sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])     # 홀수 번째 인덱스는 cos 함수 적

        return torch.FloatTensor(sinusoid_table).unsqueeze(0)   # numpy 배열을 PyTorch FloatTensor로 변환한 다음 unsqueze(0)를 사용하여 텐서에 추가 차원을 추가

    def forward(self, x):
        return x + self.pos_table[:, :x.size(1)].clone().detach()


# 위치 인코딩을 입력 텐서 x에 추가

In [ ]:
#@title 2. Scaled dot-product


In [20]:
import torch.nn.functional as F

class ScaledDotProduct_Attention(nn.Module):
    ''' Scaled Dot-Product Attention '''

    def __init__(self, temperature, attn_dropout=0.1):    # (스케일링 계수 temp, dropout 비율 attn_dropout)
        super().__init__()
        self.temperature = temperature
        self.dropout = nn.Dropout(attn_dropout)

    def forward(self, q, k, v, mask=None):

        # ① attention score 계산
        attn = torch.matmul(q / self.temperature, k.transpose(2, 3))         # (batch_size, num_heads, query_length, key_length)가

        # ② attention을 계산하지 않는 위치를 매우 작은 값으로 치환
        # - 매우작은 값은 softmax를 거칠때 0과 가까운 값이 되므로 무시한다.
        if mask is not None:
            attn = attn.masked_fill(mask == 0, -1e9)

        # ③ soft max를 이용하여 attention weight 계산
        attn = self.dropout(F.softmax(attn, dim=-1))

        # ④ 해당 분포값에 v를 곱하여 attention value를 구한다            # (batch_size, num_heads, query_length, value_dimension)을 생
        output = torch.matmul(attn, v)

        return output, attn
        # attention values, attention weight

In [ ]:
def scaled_dot_product_attention(q, k, v, mask=None):

    matmul_qk = torch.matmul(q, k.transpose(-2, -1))

    dk = torch.tensor(k.shape[-1], dtype=torch.float32)

    scaled_attention_logits = matmul_qk / torch.sqrt(dk)

    if mask is not None:
        scaled_attention_logits = scaled_attention_logits + (mask * -1e9)

    softmax = torch.softmax(scaled_attention_logits, dim=-1)

    output = torch.matmul(softmax, v)

    return output, softmax

In [ ]:
#@title 3. Multihead attention

In [21]:
class MultiHeadAttention(nn.Module):
    ''' Multi-Head Attention module '''

    def __init__(self, n_heads, d_model, d_k, d_v, device, dropout=0.1):
        super().__init__()

        assert d_model % n_heads == 0

        self.n_heads = n_heads        # 헤드(head)의 개수: 서로 다른 어텐션(attention) 컨셉의 수
        self.d_model = d_model
        self.d_k = d_k
        self.d_v = d_v

        self.w_q = nn.Linear(d_model, n_heads * d_k, bias=False)
        self.w_k = nn.Linear(d_model, n_heads * d_k, bias=False)
        self.w_v = nn.Linear(d_model, n_heads * d_v, bias=False)

        self.fc = nn.Linear(n_heads* d_v, d_model, bias=False)

        self.attention = ScaledDotProduct_Attention(temperature=d_k ** 0.5).to(device)

        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)


    def forward(self, q, k, v, mask=None):

        d_k, d_v, n_heads = self.d_k, self.d_v, self.n_heads
        sz_b, len_q, len_k, len_v = q.size(0), q.size(1), k.size(1), v.size(1)

        print("Shape of q:", q.shape)
        print("Shape of k:", k.shape)
        print("Shape of v:", v.shape)

        residual = q

        # Pass through the pre-attention projection: b x lq x (n*dv)
        # Separate different heads: b x lq x n x dv
        q = self.w_q(q).view(sz_b, len_q, n_heads, d_k) # [256, len, 8, 64]
        k = self.w_k(k).view(sz_b, len_k, n_heads, d_k) # [256, len, 8, 64]
        v = self.w_v(v).view(sz_b, len_v, n_heads, d_v) # [256, len, 8, 64]


        # Transpose for attention dot product: b x n x lq x dv
        q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2) # [256, 8, len, 64]

        if mask is not None:
            mask = mask.unsqueeze(1)   # For head axis broadcasting.

        q, attn = self.attention(q, k, v, mask=mask)    # 하나의 차원의 attention 값으로 출력되는 것

        # Transpose to move the head dimension back: b x lq x n x dv
        # Combine the last two dimensions to concatenate all the heads together: b x lq x (n*dv)
        q = q.transpose(1, 2).contiguous().view(sz_b, len_q, -1) # [256, 36, 512]
        q = self.dropout(self.fc(q))
        q += residual

        q = self.layer_norm(q)

        return q, attn


In [ ]:
#@title 4.  Position wise Feed Forward
# -> specify한 부분을 처리하는 데 용이
# d_hid: 하나의 단어에 대한 임베딩 차원
# d_in: 입력 차원
# dropout: 드롭아웃 비율


In [22]:
class Positionwise_FeedForward(nn.Module):
    ''' A two-feed-forward-layer module '''

    def __init__(self, d_in, d_hid, dropout=0.1):    # dropout 비율 바꿔가면서 해보기
        super().__init__()
        self.w_1 = nn.Linear(d_in, d_hid) # position-wise 512 -> 2048
        self.w_2 = nn.Linear(d_hid, d_in) # position-wise 2048 -> 512
        self.layer_norm = nn.LayerNorm(d_in, eps=1e-6)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):

        residual = x

        x = self.dropout(F.relu(self.w_1(x)))
        x = self.w_2(x)
        x += residual

        x = self.layer_norm(x)

        return x


In [ ]:
#@title 5. Encoder Layer
# 인코더의 셀프 어텐션: Query = Key = Value


In [23]:
# 인코더 레이어는 입력과 출력 차원이 같음 -> 트랜스포머의 인코더는 인코더 레이어를 여러 번 중첩해서 사용 가능

class EncoderLayer(nn.Module):
    ''' Compose with two layers '''

    def __init__(self, d_model, d_inner, n_heads, d_k, d_v, device, dropout=0.1):       # dropout 비율 바꿔가면서 해보기
        super(EncoderLayer, self).__init__()

        self.self_attn = MultiHeadAttention(n_heads, d_model, d_k, d_v, device, dropout=dropout)
        self.pos_ffn = Positionwise_FeedForward(d_model, d_inner, dropout=dropout)
        self.self_attn_layer_norm = nn.LayerNorm(d_model)
        self.ff_layer_norm = nn.LayerNorm(d_model)
        # self.dropout = nn.Dropout(dropout)

    def forward(self, enc_input, src_mask=None):
        enc_output, enc_slf_attn = self.self_attn(enc_input, enc_input, enc_input, mask=src_mask)
        # enc_input을 각각 q, k ,v 값으로 self-attention 수행 후 출력 텐서와 attention 가중치 텐서를 반환
        # enc_input = self.self_attn_layer_norm(enc_input + self.dropout(enc_input))         # dropout, residual connection and layer norm
        enc_output = self.pos_ffn(enc_output)                                              # position-wise feedforward
        # enc_input = self.ff_layer_norm(enc_input + self.dropout(enc_output))

        return enc_output, enc_slf_attn


In [24]:
# src_mask는 pad_idx에 대하여 mask값을 0으로 처리하는 함수로, 필요 없는 패딩 인덱스에 대해 attention 연산을 안 하겠다는 의미

def get_pad_mask(seq, pad_idx):
    return (seq != pad_idx).unsqueeze(-2)


In [ ]:
#@title 6.  Encoder Architechture
# Encoder의 구조 = token embedding + positional embedding -> a stack of N EncoderBlock -> layer norm

In [25]:
class Encoder(nn.Module):
    ''' A encoder model with self attention mechanism. '''

    def __init__(
            self, src_vocab, d_word_vec, n_layers, n_heads, d_k, d_v,
            d_model, d_inner, pad_idx, device, dropout=0.1, max_length=200, scale_emb=False):


        super().__init__()
        self.device = device

        # 모든 단어들을 embedding (고유 백터를 가진 차원으로 변경)
        self.src_word_emb = nn.Embedding(src_vocab, d_word_vec, padding_idx=pad_idx)

        # 문장의 최대 길이까지 positional encoding을 함
        self.position_enc = Positional_Encoding(d_word_vec, max_length)
        self.dropout = nn.Dropout(p=dropout)

        # multiple encoder
        # encoder layer stack으로 encoder 층을 6개로 쌓는다.
        # encoder layer : multihead attentions, feedforward로 구성
        self.layer_stack = nn.ModuleList([
            EncoderLayer(d_model, d_inner, n_heads, d_k, d_v, device, dropout=dropout)
            for _ in range(n_layers)])

        # layer_norm
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.scale_emb = scale_emb
        self.d_model = d_model

    def forward(self, src_seq, src_mask, return_attns=True):

        enc_slf_attn_list = []

        # -- Forward
        # ① word embedding
        enc_output = self.src_word_emb(src_seq)
        if self.scale_emb:
            enc_output *= self.d_model ** 0.5

        # ② positional encoding
        enc_output = self.dropout(self.position_enc(enc_output))

        # normalization
        enc_output = self.layer_norm(enc_output).to(device)

        # ③ stacked encoder layers
        for enc_layer in self.layer_stack:
            enc_output, enc_slf_attn = enc_layer(enc_output, src_mask)
            enc_slf_attn_list += [enc_slf_attn] if return_attns else []

        if return_attns:
            return enc_output, enc_slf_attn_list
        return enc_output


In [ ]:
# https://dongdong93.tistory.com/25
# https://github.com/kimwoonggon/publicservant_AI/blob/master/%EC%8B%AC%ED%99%942_%ED%8A%B8%EB%9E%9C%EC%8A%A4%ED%8F%AC%EB%A8%B8_%EC%89%BD%EA%B2%8C%EA%B5%AC%ED%98%84%ED%95%B4%EB%B3%B4%EA%B8%B0.ipynb
# https://github.com/jadore801120/attention-is-all-you-need-pytorch/blob/master/transformer/Models.py
# https://github.com/hbchen-one/Transformer-Models-from-Scratch/blob/main/Transformer_Multi30k_German_to_English.ipynb
# https://americanoisice.tistory.com/59
# https://github.com/SethHWeidman/pytorch-seq2seq/blob/master/6%20-%20Attention%20is%20All%20You%20Need.ipynb

In [ ]:
#@title Decoder Layer


In [26]:
# 인코더 layer와 마찬가지로 입력과 출력의 차원이 같음 -> 트랜스포머의 디코더 또한 디코더 레이어를 여러 번 중첩해서 사용 가능
# 두 개의 multihead attetion이 사용
# masked multi-head attention을 하기 위한 mask 과정

def get_subsequent_mask(seq):
    ''' For masking out the subsequent info. '''
    sz_b, len_s = seq.size()
    subsequent_mask = (1 - torch.triu(
        torch.ones((1, len_s, len_s), device=seq.device), diagonal=1)).bool()
    return subsequent_mask

# 대각행렬 윗부분을 False로 치환 -> 0으로 설정함으로써 미래 위치에 대한 예측이 이전에 생성된 위치만을 기반으로 함

In [27]:
class DecoderLayer(nn.Module):
    ''' Compose with three layers '''

    def __init__(self, d_model, d_inner, n_heads, d_k, d_v, device, dropout=0.1):
        super(DecoderLayer, self).__init__()

        self.self_attn = MultiHeadAttention(n_heads, d_model, d_k, d_v, device, dropout=dropout)  # decoder layer의 self-attention
        self.enc_attn = MultiHeadAttention(n_heads, d_model, d_k, d_v, device, dropout=dropout)   # decoder-encoder layer의 self-attention
        self.pos_ffn = Positionwise_FeedForward(d_model, d_inner, dropout=dropout)
        self.self_attn_layer_norm = nn.LayerNorm(d_model)
        self.ff_layer_norm = nn.LayerNorm(d_model)
        # self.dropout = nn.Dropout(dropout)

    # 인코더의 출력값을 attention하는 구조
    def forward(
            self, dec_input, enc_output,
            src_mask=None, dec_mask=None):

        # ① sublayer 1: decoder input 값, 자기 자신에 대한 self attention
        dec_output, dec_slf_attn = self.self_attn(dec_input, dec_input, dec_input, mask=src_mask)
        # dec_input = self.self_attn_layer_norm(dec_input + self.dropout(dec_output))

        # ② sublayer 2 : ①의 결과값을 query로, key,value는 encoder의 output 값으로 attention - 디코더의 query를 이용해 인코더를 attention
        dec_output, dec_enc_attn = self.enc_attn(dec_output, enc_output, enc_output, mask=dec_mask)
        # dec_input = self.ff_layer_norm(dec_input + self.dropout(dec_output))

        # ③ sublayer 3 : position wise feed forward를 통과
        dec_output = self.pos_ffn(dec_output)


        return dec_output, dec_slf_attn, dec_enc_attn


In [ ]:
#@title Decoder Architechture
#

In [28]:
from thinc.layers.dropout import Dropout
class Decoder(nn.Module):
    ''' A decoder model with self attention mechanism. '''


# trg_vocab: target 단어 수, d_word_vec: 단어 임베딩 차
    def __init__(
            self, trg_vocab, d_word_vec, n_layers, n_heads, d_k, d_v,
            d_model, d_inner, pad_idx,device, dropout=0.1, max_length=200, scale_emb=False):

        super().__init__()
        self.device = device

        # target word embedding
        self.trg_word_emb = nn.Embedding(trg_vocab, d_word_vec, padding_idx=pad_idx)

        # positional encoding
        self.position_enc = Positional_Encoding(d_word_vec, max_length=max_length)
        self.dropout = nn.Dropout(p=dropout)

         # multiple decoder
        self.layer_stack = nn.ModuleList([
            DecoderLayer(d_model, d_inner, n_heads, d_k, d_v, device, dropout=dropout)
            for _ in range(n_layers)])

        # layer_normalization
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.scale_emb = scale_emb
        self.d_model = d_model

    def forward(self, trg_seq, trg_mask, enc_output, src_mask, return_attns=True):

        dec_slf_attn_list, dec_enc_attn_list = [], []

        # -- Forward

        # ① target words embedding
        dec_output = self.trg_word_emb(trg_seq)
        if self.scale_emb:
            dec_output *= self.d_model ** 0.5

        # ② positional encoding
        dec_output = self.dropout(self.position_enc(dec_output))
        dec_output = self.layer_norm(dec_output).to(device)

        # ③ decoder_layer stacked
        for dec_layer in self.layer_stack:
            dec_output, dec_slf_attn, dec_enc_attn = dec_layer(
                dec_output, enc_output, trg_mask, src_mask)

            dec_slf_attn_list += [dec_slf_attn] if return_attns else []
            dec_enc_attn_list += [dec_enc_attn] if return_attns else []

        if return_attns:
            return dec_output, dec_slf_attn_list, dec_enc_attn_list
        return dec_output


In [57]:
#@title Transformer Model

In [29]:
# 1. source, target 데이터에 대한 mask 생성
# 2. 인코더, 디코더 층 통과
# 3. 최종 가중치 층을 통과해 출력 문장 생성

''' Define the Transformer model '''

# from transformer.Layers import EncoderLayer, DecoderLayer

class Transformer(nn.Module):
    ''' A sequence to sequence model with attention mechanism. '''

    def __init__(
            self, src_vocab, trg_vocab, src_pad_idx, trg_pad_idx, device,
            d_word_vec=512, d_model=512, d_inner=2048,
            n_layers=6, n_heads=8, d_k=64, d_v=64, max_length=200,
            trg_emb_prj_weight_sharing=True, emb_src_trg_weight_sharing=True,
            scale_emb_or_prj='prj'):

        super().__init__()
        self.device = device
        self.dropout = 0.1

        # padding index 저장
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx

        assert scale_emb_or_prj in ['emb', 'prj', 'none']
        scale_emb = (scale_emb_or_prj == 'emb') if trg_emb_prj_weight_sharing else False
        self.scale_prj = (scale_emb_or_prj == 'prj') if trg_emb_prj_weight_sharing else False
        self.d_model = d_model

        # encoder 정의
        self.encoder = Encoder(
            src_vocab=src_vocab, max_length=max_length,
            d_word_vec=d_word_vec, d_model=d_model, d_inner=d_inner,
            n_layers=n_layers, n_heads=n_heads, d_k=d_k, d_v=d_v,
            pad_idx=src_pad_idx, dropout=dropout, scale_emb=scale_emb, device=device)

        # decoder 정의
        self.decoder = Decoder(
            trg_vocab=trg_vocab, max_length=max_length,
            d_word_vec=d_word_vec, d_model=d_model, d_inner=d_inner,
            n_layers=n_layers, n_heads=n_heads, d_k=d_k, d_v=d_v,
            pad_idx=trg_pad_idx, dropout= dropout, scale_emb=scale_emb, device=device)

        # 최종 output layers 정의
        self.trg_word_prj = nn.Linear(d_model, trg_vocab, bias=False)

        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

        assert d_model == d_word_vec, \
            'To facilitate the residual connections, ' \
            'the dimensions of all module outputs shall be the same.'

        if trg_emb_prj_weight_sharing:
            # Share the weight between target word embedding & last dense layer
            self.trg_word_prj.weight = self.decoder.trg_word_emb.weight

        if emb_src_trg_weight_sharing:
            self.encoder.src_word_emb.weight = self.decoder.trg_word_emb.weight


    def forward(self, src_seq, trg_seq):

        # ① source, target 데이터에 대한 mask 생성
        src_mask = get_pad_mask(src_seq, self.src_pad_idx)
        trg_mask = get_pad_mask(trg_seq, self.trg_pad_idx) & get_subsequent_mask(trg_seq)

        # ② encoder 층을 통과
        enc_output, *_ = self.encoder(src_seq, src_mask)

        # ③ decoder 층을 통과
        dec_output, attention1, attention2 = self.decoder(trg_seq, trg_mask, enc_output, src_mask)

        # ④ 최종 weight 층을 통과
        seq_logit = self.trg_word_prj(dec_output)
        if self.scale_prj:
            seq_logit *= self.d_model ** -0.5

        return seq_logit.view(-1, seq_logit.size(2))

In [ ]:
#@title 모델 학습
d

In [30]:
class Constants:
    PAD_WORD = '<blank>' # padding token
    UNK_WORD = '<unk>' # unknown token
    BOS_WORD = '<s>' # start token
    EOS_WORD = '</s>'


In [32]:
src_vocab_size = len(SRC.vocab)
trg_vocab_size= len(TRG.vocab)
src_pad_idx = SRC.vocab.stoi[Constants.PAD_WORD]
trg_pad_idx = TRG.vocab.stoi[Constants.PAD_WORD]
trg_bos_idx = TRG.vocab.stoi[Constants.BOS_WORD]
trg_eos_idx = TRG.vocab.stoi[Constants.EOS_WORD]
d_k = 64
d_v = 64
d_model = 512
d_word_vec = 512
d_inner_hid = 2048
n_layers = 6
n_heads = 8
dropout = 0.1
proj_share_weight = True
embs_share_weight = True
batch_size=128
max_lenght = 200


In [33]:
CUDA_VISIBLE_DEVICES=-1
transformer = Transformer(
    src_vocab_size,
    trg_vocab_size,
    src_pad_idx,
    trg_pad_idx,
    device,
    d_word_vec,
    d_model,
    d_inner_hid,
    n_layers,
    n_heads,
    d_k,
    d_v).to(device)


In [34]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(transformer):,} trainable parameters')

The model has 50,921,984 trainable parameters


In [35]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

transformer.apply(initialize_weights)

Transformer(
  (encoder): Encoder(
    (src_word_emb): Embedding(13269, 512, padding_idx=1)
    (position_enc): Positional_Encoding()
    (dropout): Dropout(p=0.1, inplace=False)
    (layer_stack): ModuleList(
      (0-5): 6 x EncoderLayer(
        (self_attn): MultiHeadAttention(
          (w_q): Linear(in_features=512, out_features=512, bias=False)
          (w_k): Linear(in_features=512, out_features=512, bias=False)
          (w_v): Linear(in_features=512, out_features=512, bias=False)
          (fc): Linear(in_features=512, out_features=512, bias=False)
          (attention): ScaledDotProduct_Attention(
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (dropout): Dropout(p=0.1, inplace=False)
          (layer_norm): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
        )
        (pos_ffn): Positionwise_FeedForward(
          (w_1): Linear(in_features=512, out_features=2048, bias=True)
          (w_2): Linear(in_features=2048, out_features=512, bias

In [36]:
class ScheduledOptim():
    '''A simple wrapper class for learning rate scheduling'''

    def __init__(self, optimizer, lr_mul, d_model, n_warmup_steps):
        self._optimizer = optimizer
        self.lr_mul = lr_mul
        self.d_model = d_model
        self.n_warmup_steps = n_warmup_steps
        self.n_steps = 0


    def step_and_update_lr(self):
        "Step with the inner optimizer"
        self._update_learning_rate()
        self._optimizer.step()


    def zero_grad(self):
        "Zero out the gradients with the inner optimizer"
        self._optimizer.zero_grad()


    def _get_lr_scale(self):
        d_model = self.d_model
        n_steps, n_warmup_steps = self.n_steps, self.n_warmup_steps
        return (d_model ** -0.5) * min(n_steps ** (-0.5), n_steps * n_warmup_steps ** (-1.5))


    def _update_learning_rate(self):
        ''' Learning rate scheduling per step '''

        self.n_steps += 1
        lr = self.lr_mul * self._get_lr_scale()

        for param_group in self._optimizer.param_groups:
            param_group['lr'] = lr

In [37]:
import torch.optim as optim
n_warmup_steps = 1000
lr_mul = 0.1
optimizer = ScheduledOptim(
    optim.Adam(transformer.parameters(), betas=(0.9, 0.98), eps=1e-09),
    lr_mul, d_model, n_warmup_steps)

In [38]:
criterion = nn.CrossEntropyLoss(ignore_index=trg_pad_idx)

# 성능 평가

In [ ]:
#@title 모델 성능 평가

In [59]:
def cal_performance(pred, gold, trg_pad_idx, smoothing=False):
    ''' Apply label smoothing if needed '''

    # loss 값을 구한다
    loss = cal_loss(pred, gold, trg_pad_idx, smoothing=smoothing)

    # 전체 단어 중에 가장 값이 높은 index 검색
    pred = pred.max(1)[1]
    gold = gold.contiguous().view(-1)   # 참조 레이블

    # 정답지에서 padding index가 아닌거 조회
    non_pad_mask = gold.ne(trg_pad_idx)

    # 예측단어 중 정답을 맞춘거의 갯수
    n_correct = pred.eq(gold).masked_select(non_pad_mask).sum().item()

    # 정답 label 의 갯수
    n_word = non_pad_mask.sum().item()

    return loss, n_correct, n_word
    # loss, 맞춘갯수, 전체갯수

In [ ]:
import torch.nn.functional as F

def cal_loss(pred, gold, trg_pad_idx, smoothing=False):     # 손실 함수 계산
    ''' Calculate cross entropy loss, apply label smoothing if needed. '''

    gold = gold.contiguous().view(-1)

    if smoothing:
        eps = 0.1
        n_class = pred.size(1)

        # gold값을 pred의 shape으로 바꿔서, one-hot encoding 적용한다.
        one_hot = torch.zeros_like(pred).scatter(1, gold.view(-1, 1), 1)

        # epsilon 값에 따라 smoothing 처리한다, [0,1,0,0] → [0.03, 0.9, 0.03, 0.03]
        one_hot = one_hot * (1 - eps) + (1 - one_hot) * eps / (n_class - 1)

        # pred 값을 softmax를 이용하여 확률값으로 변환한다.
        log_prb = F.log_softmax(pred, dim=1)

        # gold에서 padding이 아닌 값의 index를 뽑는다
        non_pad_mask = gold.ne(trg_pad_idx)

        # 예측값과 smoothing된 정답값을 곱하여 loss를 산출한다.
        loss = -(one_hot * log_prb).sum(dim=1)

        # 해당 loss값에서 mask되지 않은 값을 제거하고 loss를 구한다
        loss = loss.masked_select(non_pad_mask).sum()  # average later

    else:
        # smoothing을 사용하지 않은 경우에는 cross entropy를 사용하여 loss를 구한다.
        loss = F.cross_entropy(pred, gold, ignore_index=trg_pad_idx, reduction='sum')
    return loss

In [ ]:
def patch_src(src, pad_idx):
    src = src.transpose(0, 1)
    return src


def patch_trg(trg, pad_idx):
    trg = trg.transpose(0, 1)
    trg, gold = trg[:, :-1], trg[:, 1:].contiguous().view(-1)
    return trg, gold

In [ ]:
def train_epoch(model, traindata, optimizer, opt, device, smoothing):
    ''' Epoch operation in training phase'''

    # model.train() 학습할때 필요한 drop out, batch_normalization 등의 기능을 활성화
    # model.eval()과 model.train()을 병행하므로, 모델 학습시에는 model.train() 호출해야함
    model.train()
    total_loss, n_word_total, n_word_correct = 0, 0, 0

    desc = '  - (Training)   '
    for batch in tqdm(train_data, mininterval=2, desc=desc, leave=False):

        # prepare data
        # ① src_seq. trg_seq, trg에 대한 정답 label "gold" 생성
        src_seq = patch_src(batch.src, opt.src_pad_idx).to(device)
        trg_seq, gold = map(lambda x: x.to(device), patch_trg(batch.trg, opt.trg_pad_idx))

        # forward
        # backward 전 optimizer의 기울기를 초기화해야만 새로운 가중치 편향에 대해서 새로운 기울기를 구할 수 있습니다.
        optimizer.zero_grad()

        # ② model 예측값 생성
        pred = model(src_seq, trg_seq) # 256 * (trg 문장길이-1), 10077(vocab)

        # ③ loss값 계산
        loss, n_correct, n_word = cal_performance(
        pred, gold, opt.trg_pad_idx, smoothing=smoothing)

        # ④ parameter update 진행
        loss.backward()
        optimizer.step_and_update_lr()

        # note keeping
        n_word_total += n_word
        n_word_correct += n_correct
        total_loss += loss.item()

    # 평균 loss
    loss_per_word = total_loss/n_word_total

    # 평균 acc
    accuracy = n_word_correct/n_word_total

    return loss_per_word, accuracy

In [ ]:
def eval_epoch(model, valid_data, device, opt):
    ''' Epoch operation in evaluation phase '''

    model.eval()
    total_loss, n_word_total, n_word_correct = 0, 0, 0

    desc = '  - (Validation) '
    with torch.no_grad():
        for batch in tqdm(valid_data, mininterval=2, desc=desc, leave=False):

            # prepare data
            src_seq = patch_src(batch.src, opt.src_pad_idx).to(device)
            trg_seq, gold = map(lambda x: x.to(device), patch_trg(batch.trg, opt.trg_pad_idx))

            # forward
            pred = model(src_seq, trg_seq)
            loss, n_correct, n_word = cal_performance(
                pred, gold, opt.trg_pad_idx, smoothing=False)

            # note keeping
            n_word_total += n_word
            n_word_correct += n_correct
            total_loss += loss.item()

    loss_per_word = total_loss/n_word_total
    accuracy = n_word_correct/n_word_total
    return loss_per_word, accuracy

In [ ]:
!pip install tensorboard

In [ ]:
import os
import math
import time
from torch.utils.tensorboard import SummaryWriter

def train(model, train_data, valid_data, optimizer, device, opt):
    ''' Start training '''

    def print_performances(header, ppl, accu, start_time, lr):
        print('  - {header:12} ppl: {ppl: 8.5f}, accuracy: {accu:3.3f} %, lr: {lr:8.5f}, '\
              'elapse: {elapse:3.3f} min'.format(
                  header=f"({header})", ppl=ppl,
                  accu=100*accu, elapse=(time.time()-start_time)/60, lr=lr))

    #valid_accus = []
    valid_losses = []
    opt.epoch = 10
    for epoch_i in range(opt.epoch):
        print('[ Epoch', epoch_i, ']')

        start = time.time()
        opt.label_smoothing = 0.1
        train_loss, train_accu = train_epoch(
            model, train_data, optimizer, opt, device, smoothing=opt.label_smoothing)
        train_ppl = math.exp(min(train_loss, 100)) # train_loss : loss_per_word, PPL = exp(cross_entropy)
        # Current learning rate
        lr = optimizer._optimizer.param_groups[0]['lr']
        print_performances('Training', train_ppl, train_accu, start, lr)

        start = time.time()
        valid_loss, valid_accu = eval_epoch(model, valid_data, device, opt)
        valid_ppl = math.exp(min(valid_loss, 100))
        print_performances('Validation', valid_ppl, valid_accu, start, lr)

        valid_losses += [valid_loss]

        checkpoint = {'epoch': epoch_i, 'settings': opt, 'model': model.state_dict()}

        if opt.save_mode == 'all':
            model_name = 'model_accu_{accu:3.3f}.chkpt'.format(accu=100*valid_accu)
            torch.save(checkpoint, model_name)
        elif opt.save_mode == 'best':
            model_name = 'model.chkpt'
            if valid_loss <= min(valid_losses):
                torch.save(checkpoint, os.path.join(opt.output_dir, model_name))
                print('    - [Info] The checkpoint file has been updated.')

In [ ]:
from torchtext.datasets import TranslationDataset

def prepare_dataloaders_from_bpe_files(opt, device):
    batch_size = opt.batch_size
    MIN_FREQ = 2
    if not opt.embs_share_weight:
        raise

    data = pickle.load(open(opt.data_pkl, 'rb'))
    MAX_LEN = data['settings'].max_len
    field = data['vocab']
    fields = (field, field)

    def filter_examples_with_length(x):
        return len(vars(x)['src']) <= MAX_LEN and len(vars(x)['trg']) <= MAX_LEN

    train = TranslationDataset(
        fields=fields,
        path=opt.train_path,
        exts=('.src', '.trg'),
        filter_pred=filter_examples_with_length)
    val = TranslationDataset(
        fields=fields,
        path=opt.val_path,
        exts=('.src', '.trg'),
        filter_pred=filter_examples_with_length)

    opt.max_token_seq_len = MAX_LEN + 2
    opt.src_pad_idx = opt.trg_pad_idx = field.vocab.stoi[Constants.PAD_WORD]
    opt.src_vocab_size = opt.trg_vocab_size = len(field.vocab)

    train_iterator = BucketIterator(train, batch_size=batch_size, device=device, train=True)
    val_iterator = BucketIterator(val, batch_size=batch_size, device=device)
    return train_iterator, val_iterator


In [ ]:
from torchtext.data import Field, Dataset, BucketIterator

def prepare_dataloaders(opt, device):
    batch_size = opt.batch_size
    data = pickle.load(open(opt.data_pkl, 'rb'))

    opt.max_token_seq_len = data['settings'].max_len
    opt.src_pad_idx = data['vocab']['src'].vocab.stoi[Constants.PAD_WORD]
    opt.trg_pad_idx = data['vocab']['trg'].vocab.stoi[Constants.PAD_WORD]

    opt.src_vocab_size = len(data['vocab']['src'].vocab)
    opt.trg_vocab_size = len(data['vocab']['trg'].vocab)

    if opt.embs_share_weight:
      assert data['vocab']['src'].vocab.stoi == data['vocab']['trg'].vocab.stoi, \
          'To sharing word embedding the src/trg word2idx table shall be the same.'

    fields = {'src': data['vocab']['src'], 'trg':data['vocab']['trg']}

    train = Dataset(examples=data['train'], fields=fields)
    val = Dataset(examples=data['valid'], fields=fields)

    train_iterator = BucketIterator(train, batch_size=batch_size, device=device, train=True)
    val_iterator = BucketIterator(val, batch_size=batch_size, device=device)

    return train_iterator, val_iterator


In [ ]:
#@title translate task


In [ ]:
''' Translate input text with trained model. '''

import torch
import dill as pickle
from tqdm import tqdm

def load_model(opt, device):

    # load model
    checkpoint = torch.load(opt.model, map_location=device)

    # model의 option load
    model_opt = checkpoint['settings']

    # transformer model을 model option에 따라 재생성
    model = Transformer(
        model_opt.src_vocab_size,
        model_opt.trg_vocab_size,
        model_opt.src_pad_idx,
        model_opt.trg_pad_idx,
        trg_emb_prj_weight_sharing=model_opt.proj_share_weight,
        emb_src_trg_weight_sharing=model_opt.embs_share_weight,
        d_k=model_opt.d_k,
        d_v=model_opt.d_v,
        d_model=model_opt.d_model,
        d_word_vec=model_opt.d_word_vec,
        d_inner=model_opt.d_inner_hid,
        n_layers=model_opt.n_layers,
        n_head=model_opt.n_head,
        dropout=model_opt.dropout).to(device)

    # 학습된 weight를 model에 반영
    model.load_state_dict(checkpoint['model'])
    print('[Info] Trained model state loaded.')
    return model

In [ ]:
import time
import math
import random

model = load_model(opt, device)
N_EPOCHS = 10
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(transformer, train_data, valid_data, optimizer, device, opt)

    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')

AttributeError: ignored

# 성능 평가 2

In [39]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()
        output = model(src, trg[:, :-1])
        output_reshape = output.contiguous().view(-1, output.shape[-1])
        trg = trg[:, 1:].contiguous().view(-1)

        loss = criterion(output_reshape, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        epoch_loss += loss.item()
        print('step :', round((i / len(iterator)) * 100, 2), '% , loss :', loss.item())

    return epoch_loss / len(iterator)

In [40]:
from torchtext.data.metrics import bleu_score
candidate_corpus = [['My', 'full', 'pytorch', 'test'], ['Another', 'Sentence']]
references_corpus = [[['My', 'full', 'pytorch', 'test'], ['Completely', 'Different']], [['No', 'Match']]]
bleu_score(candidate_corpus, references_corpus)

0.8408964276313782

In [41]:
from torchtext.data.metrics import bleu_score

def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    batch_bleu = []
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg
            output = model(src, trg[:, :-1])
            output_reshape = output.contiguous().view(-1, output.shape[-1])
            trg = trg[:, 1:].contiguous().view(-1)

            loss = criterion(output_reshape, trg)
            epoch_loss += loss.item()

            trg_words = [[TRG.target.vocab.itos[idx] for idx in trg[i].tolist()] for i in range(trg.shape[0])]
            output_words = [[TRG.target.vocab.itos[idx] for idx in output[i].max(dim=1)[1].tolist()] for i in range(output.shape[0])]

            bleu = bleu_score(output_words, trg_words)
            batch_bleu.append(bleu)

    batch_bleu = sum(batch_bleu) / len(batch_bleu)
    return epoch_loss / len(iterator), batch_bleu

In [42]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [43]:
clip = 1.0
import time

def run(total_epoch, best_loss):
    train_losses, test_losses, bleus = [], [], []
    for step in range(total_epoch):
        start_time = time.time()
        train_loss = train(transformer, train_iterator, optimizer, criterion, clip)
        valid_loss, bleu = evaluate(transformer, valid_iterator, criterion)
        end_time = time.time()

        if step > n_warmup_steps:
            ScheduledOptim.step(valid_loss)

        train_losses.append(train_loss)
        test_losses.append(valid_loss)
        bleus.append(bleu)
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        if valid_loss < best_loss:
            best_loss = valid_loss
            torch.save(transformer.state_dict(), 'saved/model-{0}.pt'.format(valid_loss))

        f = open('result/train_loss.txt', 'w')
        f.write(str(train_losses))
        f.close()

        f = open('result/bleu.txt', 'w')
        f.write(str(bleus))
        f.close()

        f = open('result/test_loss.txt', 'w')
        f.write(str(test_losses))
        f.close()

        print(f'Epoch: {step + 1} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
        print(f'\tVal Loss: {valid_loss:.3f} |  Val PPL: {math.exp(valid_loss):7.3f}')
        print(f'\tBLEU Score: {bleu:.3f}')


In [44]:
factor = 0.9
adam_eps = 5e-9
patience = 10
epoch = 1000
weight_decay = 5e-4
inf = float('inf')

In [46]:
run(total_epoch=epoch, best_loss=inf)


Shape of q: torch.Size([24, 128, 512])
Shape of k: torch.Size([24, 128, 512])
Shape of v: torch.Size([24, 128, 512])


RuntimeError: ignored